We have a bronze table being loaded into our data lake using a third-party tool. There has been a request to clean up the data and resolve known issues. Your task is to write the needed Python code that will address each of the following issues.

The following are the issues present:

Wrong column name: The date column is spelled wrong
Nulls not correctly identified: The sales_id column has null values as NA strings
Data with missing values is unwanted: Any data with a null in sales_id should be dropped
Duplicate sales_id: Take the first value of any duplicate rows
Date column not DateType: The date column is not a DateType


In [2]:
from functools import partial
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, regexp_replace, flatten, explode, struct, create_map, array
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType, TimestampType

In [4]:
spark = SparkSession.builder.appName('chap-2').master("local[*]").getOrCreate()

In [5]:
bronze_sales = spark.createDataFrame(data = [
    ("1", "LA", "2000-01-01",5, 1400),
    ("2", "LA", "1998-2-01",4, 1500),
    ("2", "LA", "1998-2-01",4, 1500),
    ("3", "LA", "1997-4-01",6, 1300),
    ("4", "LA", "2005-5-01",2, 1100),
    ("NA", "LA", "2013-6-01",1, 1200),
  ], schema = ["sales_id", "city", "dat"," clerk_id", "total_sales"])

In [6]:
#Renaming the date column, here i just create a new df with date column renamed and dat column dropped.

one = bronze_sales.select(col("*"),col("dat").alias("date")).drop("dat")

In [7]:
two = bronze_sales.select(col("*"),when(col('sales_id')== "NA", None).otherwise(col("sales_id")).alias("Cleaned_sales_id"))\
.drop("sales_id").select(col("*"),col("cleaned_sales_id").alias("sales_id")) \
.drop("cleaned_sales_id")

In [8]:
two.show()

+----+----------+---------+-----------+--------+
|city|       dat| clerk_id|total_sales|sales_id|
+----+----------+---------+-----------+--------+
|  LA|2000-01-01|        5|       1400|       1|
|  LA| 1998-2-01|        4|       1500|       2|
|  LA| 1998-2-01|        4|       1500|       2|
|  LA| 1997-4-01|        6|       1300|       3|
|  LA| 2005-5-01|        2|       1100|       4|
|  LA| 2013-6-01|        1|       1200|    NULL|
+----+----------+---------+-----------+--------+



In [9]:
three = two.na.drop(subset=["sales_id"])
three.show()

+----+----------+---------+-----------+--------+
|city|       dat| clerk_id|total_sales|sales_id|
+----+----------+---------+-----------+--------+
|  LA|2000-01-01|        5|       1400|       1|
|  LA| 1998-2-01|        4|       1500|       2|
|  LA| 1998-2-01|        4|       1500|       2|
|  LA| 1997-4-01|        6|       1300|       3|
|  LA| 2005-5-01|        2|       1100|       4|
+----+----------+---------+-----------+--------+



In [10]:
four = three.dropDuplicates(subset=["sales_id"])

In [11]:
four.show()

+----+----------+---------+-----------+--------+
|city|       dat| clerk_id|total_sales|sales_id|
+----+----------+---------+-----------+--------+
|  LA|2000-01-01|        5|       1400|       1|
|  LA| 1998-2-01|        4|       1500|       2|
|  LA| 1997-4-01|        6|       1300|       3|
|  LA| 2005-5-01|        2|       1100|       4|
+----+----------+---------+-----------+--------+



In [12]:
from pyspark.sql.functions import to_date
five = one.select(col("*"),to_date("date").alias("date_fixed"))\
.drop("date") \
.select(col("date_fixed").alias("date"))\
.drop("date_fixed")\
.show()

+----------+
|      date|
+----------+
|2000-01-01|
|1998-02-01|
|1998-02-01|
|1997-04-01|
|2005-05-01|
|2013-06-01|
+----------+

